In [0]:
%sql
CREATE TABLE workspace.stream.sourcetable
(
  color STRING
)
USING DELTA

In [0]:
from delta.tables import DeltaTable

In [0]:
DeltaTable.createIfNotExists(spark)\
    .tableName("workspace.stream.sourcetable") \
    .addColumn("color", "STRING") \
    .execute()

In [0]:
DeltaTable.createIfNotExists(spark)\
    .tableName("workspace.stream.sinktable") \
    .addColumn("color", "STRING") \
    .execute()

In [0]:
%sql
INSERT INTO workspace.stream.sourcetable VALUES 
('red'),
 ('green'), 
 ('blue'), 
 ('yellow'), 
 ('orange'), 
 ('orange')

In [0]:
%sql
SELECT * FROM workspace.stream.sourcetable

In [0]:
df = spark.readStream.table('workspace.stream.sourcetable')

In [0]:
from pyspark.sql.functions import *

In [0]:
df = df.groupBy("color").agg(count("*").alias("count"))

In [0]:
df.writeStream.format("delta") \
        .outputMode('complete') \
        .trigger(once=True) \
        .option("checkpointLocation", "/Volumes/workspace/stream/streaming/output/check") \
        .option("path", "/Volumes/workspace/stream/streaming/output/Data") \
        .start()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/stream/streaming/output/Data`